# About
Implementation of the 5-Star Ratings Problem

In [2]:
import numpy as np

Functions:

In [8]:
def f(x):
    return 2 * x

def is_possible_scenario(x):
    
    # Placeholder response
    possible_flags = (x > 0.5).astype(int)
    
    return possible_flags 

In [ ]:
# Define the function g which returns 1 if the element is greater than 0.5, else 0
def g(x):
    return (x > 0.5).astype(int)

# Apply f to A
B = f(A)

# Apply g to A
C = g(A)

# Element-wise multiplication of B and C
D = B * C

# Sum of the elements of D
sum_D = np.sum(D)
print("Sum of the elements in D:", sum_D)

# MAIN

Settings:

In [70]:
average_rating = 4.8

n_trials = 5

dimensions = 2  # **
# **Come back and think about whether it is ok to call this "dimensions".
#   If you know n-1 faces you know the final face too...
#   But for this method it is more intuitive to give one dimension per face anyways

Derived variables:

In [71]:
array_shape = [n_trials for _ in range(dimensions)]
print("Array Shape:\n", shape)

Array Shape:
 [12, 12, 12, 12, 12]


In [72]:
# Array with one-based-indexing
index_array = np.arange(n_trials) + 1
index_array

array([1, 2, 3, 4, 5])

### Starting Matrices:

Index position matrix:
* Looks like using `np.indices` is the trick here
* Can I make this by matrix multiplication?
* Is this needed?
    * Think about whether we need this matrix or can "skip it" and use maybe a placeholder with the required shape

In [90]:
import numpy as np

def create_ndim_array(num_dims, size_per_dim):
    """
    Create an N-dimensional NumPy array where each element is the sum of its indices.

    :param num_dims: Number of dimensions of the array
    :param size_per_dim: Size of each dimension
    :return: N-dimensional array with each element being the sum of its indices
    """
    # Generate arrays of indices for each dimension
    indices = np.indices((size_per_dim,) * num_dims)
    
    # Sum along the first axis to sum across all dimensions
    array_sum = indices.sum(axis=0)
    return array_sum

# Example usage
num_dims = 2  # Number of dimensions
size_per_dim = 5  # Size of each dimension

ndim_array = create_ndim_array(num_dims, size_per_dim)
print(ndim_array)
print("Shape of the array:", ndim_array.shape)


[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [4 5 6 7 8]]
Shape of the array: (5, 5)


Roughwork to understand:

In [99]:
indices = np.indices((5,) * 2)  # (n_trials,) * n_dimensions
indices

array([[[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]]])

In [ ]:
indices.sum(axis=1)

In [101]:
indices.sum(axis=0)

array([[10, 10, 10, 10, 10],
       [ 0,  5, 10, 15, 20]])

Feasible outcome matrix 
* A boolean matrix
* True if sum of indices = `n_trials`

### Matrix Operations 